In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files


TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU


In [ ]:
!nvidia-smi

Sat Jul  3 19:33:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2



In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 263Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.72Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 450Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:24, 20.4Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 874Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.35Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 4.20Mit/s]


## Mounting Google Drive



In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1VE2L7AuuR2EkLBZiX9P8ybVTBjHsSsKT'
output = 'my_archive.tar'
gdown.download(url, output, quiet=False)
!unzip -q 'my_archive.tar'



Downloading...
From: https://drive.google.com/uc?id=1VE2L7AuuR2EkLBZiX9P8ybVTBjHsSsKT
To: /content/my_archive.tar
2.34MB [00:00, 168MB/s]


In [ ]:
from collections import defaultdict
from pathlib import Path
import pandas as pd

my_dir_path = "results/job_hero/"

results = defaultdict(list)
for file in Path(my_dir_path).iterdir():
    with open(file, "r") as file_open:
        results["title"].append(file)
        results["body"].append(file_open.read())
    df = pd.DataFrame(results)




In [ ]:
df =df.astype(str)
df["title"] = df["title"].replace('.txt', '', regex=True)
df["title"] = df["title"].replace('results/job_hero/', '', regex=True)
df.head()

,title,body
0,Zoo Keeper,Dear Ms. Rogers:\nWhen I learned of the Portla...
1,Certified Medical Assistant,Dear Mr. Brown:\nAs a skilled medical professi...
2,Director Of Development,Dear Mr. Trammell:\nWhen I learned of your sea...
3,Sales Trainer,Dear Mr. Montelongo:\nAs a respected and exper...
4,Customer Service Manager,"Dear Ms. Brown:\nWith the enclosed resume, I w..."


In [1]:
# !pip install textract
# !pip install jsonlines

In [ ]:
import os
import textract
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
from spacy.lang.en import English
from spacy.tokens import Doc
import spacy


nlp = spacy.load('en')
skill_pattern_path = "jz_skill_patterns.jsonl"
with jsonlines.open("jz_skill_patterns.jsonl") as f:
    created_entities = [line['label'].upper() for line in f.iter()]

ruler = EntityRuler(nlp).from_disk(skill_pattern_path)
nlp.add_pipe(ruler, after='parser')

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_=="SKILL":
             subset.append(ent.text)
             subset = list(dict.fromkeys(subset))
    myset.append(subset)
    return subset

In [ ]:
df['skills'] = df['body'].apply(get_skills)

In [ ]:
df.head()

,start,title,skills,body,end
0,<|startoftext|>,Zoo Keeper,"[monitoring, marketing, communications]",Dear Ms. Rogers:\nWhen I learned of the Portla...,<|endoftext|>
1,<|startoftext|>,Certified Medical Assistant,"[monitoring, support]",Dear Mr. Brown:\nAs a skilled medical professi...,<|endoftext|>
2,<|startoftext|>,Director Of Development,"[marketing, support, business]",Dear Mr. Trammell:\nWhen I learned of your sea...,<|endoftext|>
3,<|startoftext|>,Sales Trainer,"[business, support, marketing, Monitoring]",Dear Mr. Montelongo:\nAs a respected and exper...,<|endoftext|>
4,<|startoftext|>,Customer Service Manager,"[support, communications]","Dear Ms. Brown:\nWith the enclosed resume, I w...",<|endoftext|>


In [ ]:
df['start'] = '<|startoftext|>'
df['end'] = '<|endoftext|>'

# df.skills = list(set(df.skills))

# skills = list(dict.fromkeys(skills))


# separator = ', '
# df['skills'] = separator.join(df['skills'])

df['skills'] = [', '.join(map(str, l)) for l in df['skills']]
df['skills'] = df["skills"].map(lambda x: x.lower())
df=df[['start', 'title', 'skills','body', 'end']]
df.head()

,start,title,skills,body,end
0,<|startoftext|>,Zoo Keeper,"monitoring, marketing, communications",Dear Ms. Rogers:\nWhen I learned of the Portla...,<|endoftext|>
1,<|startoftext|>,Certified Medical Assistant,"monitoring, support",Dear Mr. Brown:\nAs a skilled medical professi...,<|endoftext|>
2,<|startoftext|>,Director Of Development,"marketing, support, business",Dear Mr. Trammell:\nWhen I learned of your sea...,<|endoftext|>
3,<|startoftext|>,Sales Trainer,"business, support, marketing, monitoring",Dear Mr. Montelongo:\nAs a respected and exper...,<|endoftext|>
4,<|startoftext|>,Customer Service Manager,"support, communications","Dear Ms. Brown:\nWith the enclosed resume, I w...",<|endoftext|>


In [ ]:
df.to_csv(r'cover_letters_titles.txt', header=None, index=None, sep='~', mode='a')

## Uploading a Text File to be Trained to Colaboratory


In [ ]:
file_name='cover_letters_titles.txt'

## Training GPT-2



In [ ]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1700,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


dataset has 616325 tokens
Training...
[10 | 27.72] loss=2.69 avg=2.69
[20 | 49.55] loss=2.44 avg=2.57
[30 | 71.97] loss=2.40 avg=2.51
[40 | 95.09] loss=2.19 avg=2.43
[50 | 119.18] loss=2.04 avg=2.35
[60 | 142.59] loss=2.27 avg=2.34
[70 | 165.73] loss=2.22 avg=2.32
[80 | 189.08] loss=2.13 avg=2.29
[90 | 212.63] loss=2.03 avg=2.26
[100 | 236.13] loss=1.96 avg=2.23
[110 | 259.52] loss=1.91 avg=2.20
[120 | 282.87] loss=1.96 avg=2.18
[130 | 306.26] loss=1.89 avg=2.16
[140 | 329.74] loss=1.85 avg=2.13
[150 | 353.28] loss=1.90 avg=2.12
[160 | 376.75] loss=1.85 avg=2.10
[170 | 400.20] loss=1.79 avg=2.08
[180 | 423.63] loss=1.91 avg=2.07
[190 | 447.02] loss=1.78 avg=2.05
[200 | 470.40] loss=1.79 avg=2.04
======== SAMPLE 1 ========
 attendance, my background in telecommunications, combined with my leadership and leadership talents, has prepared me to excel in this role.
My proven success in managing both new and existing telephone lines while ensuring full coverage of busy areas such as roads, s

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint



In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1700
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1700


## Generate Text From The Trained Model



In [ ]:
??gpt2.generate

In [ ]:
gpt2.generate(sess,
              temperature=0.7,
              top_p=0.9,
              nsamples=3,
              batch_size=3,
              length=700,
              prefix="~Data Scientist~machine learning, sql, algorithms, visualization~Dear Ms. Mustermann",
              truncate="<|endoftext|>",
              include_prefix=False
              )

:
When I learned of your search for a Data Scientist, I felt compelled to submit my resume for your consideration. As a skilled and detail-oriented professional with a solid background in data science and knowledge of statistical models, I am confident that I would be a valuable asset to your team.
My background includes collecting and processing large data sets, manipulating and analyzing data, and working effectively with cross-functional staff to drive research project success. With a solid commitment to ensuring data integrity and confidentiality, I excel at managing projects, overseeing lab staff, and ensuring top-flight quality control.
Highlights of my experience include…
 Overseeing data management and quality control for Pediatric & Adult Child Cohort, using machine learning and algorithms to create and implement thoughtful and scientific programs to improve outcomes
 Creating and updating standardization campaigns for large datasets, using social media tools as well as resear

# Bulk generation

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1